In [118]:
import numpy as np
from sionna.rt import load_scene, PlanarArray, Transmitter, Camera, Receiver, RadioMaterial
import csv

In [119]:
import matplotlib.pyplot as plt

In [120]:
colab_compat = False
try:
    import google.colab
    colab_compat = True
except:
    colab_compat = False
    

In [121]:
# By default there are some scenes available in the sionna 
# they can be loaded as - sionna.rt.scene.etiole
scene = load_scene("without_leather/modifiedv2_without_leather.xml")

In [122]:
def addCamera(position,look_at,name):
    my_cam = Camera(name, position = position, look_at = look_at)
    return my_cam

In [123]:
def addPlanarArray(rows, cols, v_spacing = 0.5, h_spacing = 0.7, patt = "tr38901", polarization = "V"):
    return PlanarArray(num_rows = rows,
                      num_cols = cols,
                      vertical_spacing=v_spacing,
                      horizontal_spacing=h_spacing,
                      pattern=patt,
                      polarization=polarization)


In [124]:
# we can print available cameras for the scenes 
# These cameras are added when this scene was created in the blender or in any other software
# We can add our own camera in sionna for that we need its location and where it is looking at both coordinates are needed
# To add anything in the scene use scene.add(var_name)
print(scene.cameras)

{'scene-cam-0': <sionna.rt.camera.Camera object at 0x7786687efbd0>}


In [ ]:
if colab_compat:
    scene.render(camera='scene-cam-0')
scene.preview()

In [126]:
# Listing all the objects and their materials 
objects = []
for i , obj in enumerate(scene.objects.values()):
    objects.append([obj.name,obj.radio_material.name])
print(objects[:7])
for name, material in objects:
    if(material == 'rubber'):
        print(f"rubber is available for {name} object")

[['Cube', 'itu_wood'], ['Cube_1', 'itu_wood'], ['TV', 'itu_glass'], ['W_rfel', 'itu_glass'], ['W_rfel_1', 'itu_glass'], ['W_rfel_2', 'itu_glass'], ['W_rfel_10', 'itu_glass']]


In [127]:
# configure antenna array for all transmitters
scene.tx_array = addPlanarArray(2,2)
# configure antenna array for all receivers
scene.rx_array = addPlanarArray(1,1,0.5,0.5,"dipole","cross")

In [ ]:
print(scene.tx_array.show())
print(scene.rx_array.show())

In [129]:
# While exporting via mitsuba in blender check save_id option 
# Forward should be Y forward
# Up should be Z Up
tx = Transmitter(name = "tx", position = [245,-131,295]) #tv's right side wall  
rx = Receiver(name = "rx", position = [-244,140,100]) #tv's left side wall
scene.add(tx)
scene.add(rx)
tx.look_at(rx) # Transmitter points towards receiver

In [130]:
scene.frequency = 2.14e9 # in Hz, implicitly updates RadioMaterials
scene.synthetic_array = True # if set to False, ray tracing will be done per antenna element (slower for larger array)

In [131]:
def matt(f_hz):
    return (compute_relative_permittivity(f_hz), compute_conductivity(f_hz))


<span style="color:yellow">  The parameter``` max_depth ``` determines the maximum number of interactions between a ray and a scene objects </span>. For example, with a ``` max_depth ```of one, only LoS paths are considered. When the property``` scene.synthetic_array ```is set to ```False```, antenna arrays are explicitly modeled by finding paths between any pair of transmitting and receiving antennas in the scene. Otherwise, arrays are represented by a single antenna located in the center of the array. Phase shifts related to the relative antenna positions will then be applied based on a plane-wave assumption when the channel impulse responses are computed.
<a href="https://nvlabs.github.io/sionna/api/rt.html#sionna.rt.Scene.compute_paths">Compute Paths API</a>

In [132]:
paths = scene.compute_paths(max_depth=15,
                            num_samples=1e6,scattering=True,diffraction=True) 

In [ ]:
scene.preview(paths, show_devices=True, show_paths=True) 

In [134]:

# Show the types of all paths:
# 0 - LoS, 1 - Reflected, 2 - Diffracted, 3 - Scattered
# Note that Diffraction and scattering are turned off by default.
path_types = np.array([0,0,0,0])
for i in paths.types.numpy():
    for n in i:
        path_types[n] += 1
print(path_types)

[  1  43  18 970]


In [136]:
paths.a.shape

TensorShape([1, 1, 2, 1, 4, 1032, 1])

In [137]:
# Default parameters in the PUSCHConfig
subcarrier_spacing = 15e3
fft_size = 48


In [138]:
paths.apply_doppler(sampling_frequency=subcarrier_spacing, # Set to 15e3 Hz
                    num_time_steps=14, # Number of OFDM symbols
                    tx_velocities=[3.,0,0], # We can set additional tx speeds
                    rx_velocities=[0,7.,0]) # Or rx speeds

In [139]:
paths.a.shape

TensorShape([1, 1, 2, 1, 4, 1032, 14])

In [140]:
a, tau = paths.cir()
print("Shape of tau: ", tau.shape)


Shape of tau:  (1, 1, 1, 1032)


In [ ]:
t = tau[0,0,0,:]/1e-9 # Scale to ns
a_abs = np.abs(a)[0,0,0,0,0,:,0]
a_max = np.max(a_abs)
# Add dummy entry at start/end for nicer figure
t = np.concatenate([(0.,), t, (np.max(t)*1.1,)])
a_abs = np.concatenate([(np.nan,), a_abs, (np.nan,)])

# And plot the CIR
plt.figure()
plt.title("Channel impulse response realization")

plt.stem(t, a_abs)
plt.xlim([0, np.max(t)])
plt.ylim([-2e-6, a_max*1.1])
plt.xlabel(r"$\tau$ [ns]")
plt.ylabel(r"$|a|$");